In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

# !pip install autoreject
import mne
from autoreject import AutoReject

In [5]:


# Python file because jupyter doesn't work with the interactive plots
#Loads a dataset from a CSV file into an mne raw eeg object

columns = [1,2,3,4,5,6,7,8]

with open(r"/Users/mercy/Downloads/eeg/dataset_creation/datasets/1_MB/1_MB_1.csv") as f:
    rawarr = np.loadtxt(f,usecols = columns,skiprows = 1,delimiter=',')

# Convert from microvolts to volts
rawarr = rawarr / (1_000_000)

print(rawarr)

# channel_names = ['Fs2','Fs1','C4','C3','T6','T5','P4','P3']
channel_names = ['EXG Channel 0',
                 'EXG Channel 1',
                 'EXG Channel 2',
                 'EXG Channel 3',
                 'EXG Channel 4',
                 'EXG Channel 5',
                 'EXG Channel 6',
                 'EXG Channel 7',
                 ]
channel_types = 'eeg'
sample_rate = 250

inf = mne.create_info(ch_names=channel_names,ch_types=channel_types,sfreq=sample_rate)

raw = mne.io.RawArray(np.transpose(rawarr), info=inf,first_samp=100)

# Filter settings
low_cut = 0.1 
ica_low_cut = 1.0 

hi_cut  = 30

# ICA settings
seed = 42
ica_n_components = .999

# EOG Channel names
# Assume channels are close enough to the eyes to act as EOG channels
EOG_ch_names = ['EXG Channel 0','EXG Channel 1']

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.04352589  0.04763988 -0.0496261  ... -0.01922053 -0.00990303
  -0.01729996]
 [ 0.04353485  0.04765222 -0.04961888 ... -0.01922315 -0.00990272
  -0.01730251]
 ...
 [ 0.04427675  0.04846627 -0.04815909 ... -0.01831775 -0.01589426
  -0.01662205]
 [ 0.04430158  0.04848603 -0.04813151 ... -0.01829235 -0.01587019
  -0.01659494]
 [ 0.04427988  0.0484601  -0.04813933 ... -0.01829524 -0.0158802
  -0.01660196]]
Creating RawArray with float64 data, n_channels=8, n_times=54094
    Range : 100 ... 54193 =      0.400 ...   216.772 secs
Ready.


In [6]:


# Normalize, filter, and reject artefacts from data
def clean(raw):
    raw.load_data()

    # Normalize the input data

    # Regular filter in the 0.1-30hz range
    raw = raw.filter(low_cut, hi_cut)

    # ICA based artefact rejection
    run_ICA(raw,use_autoreject=False)


def run_ICA(data, use_autoreject=False): 

    # Heavy highpass for ICA training data
    train_data = raw.copy().filter(ica_low_cut, None)

    ica = mne.preprocessing.ICA(n_components=ica_n_components,
                                random_state=seed,
                                )

    # Autoreject epoch extraction
    if use_autoreject:
        train_data = do_autoreject(train_data)
    
    ica.fit(train_data)

    # Use EOG signal to eliminate eye blink
    
    eog_indices, _ = ica.find_bads_eog(raw,EOG_ch_names)
    ica.exclude = eog_indices

    # Try to find muscle artefacts

    muscle_indices, _ = ica.find_bads_muscle(raw)
    ica.exclude.extend(muscle_indices)


    # Reconstruct data
    reconst_raw = raw.copy()
    ica.apply(reconst_raw)

    raw.plot(clipping=None)
    plt.show()

    reconst_raw.plot(clipping=None)
    plt.show()

# Applies autoreject to prevent large artefacts from affecting the ICA training
def do_autoreject(train_data):
    # Break data into 1 s epochs
        tstep = 1.0
        events_ica = mne.make_fixed_length_events(train_data, duration=tstep)
        epochs_ica = mne.Epochs(train_data, events_ica,
                                tmin=0.0, tmax=tstep,
                                baseline=None,
                                preload=True)
        

        
        ar = AutoReject(n_interpolate=[1, 2, 4],
                    random_state=42,
                    picks=mne.pick_types(epochs_ica.info, 
                                        eeg=True,
                                        eog=False
                                        ),
                    n_jobs=-1, 
                    verbose=False
                    )

        ar.fit(epochs_ica)

        reject_log = ar.get_reject_log(epochs_ica)
        return epochs_ica[~reject_log.bad_epochs]


In [7]:

clean(raw)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 8251 samples (33.004 s)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 825 samples

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s finished


Selecting by explained variance: 2 components
Fitting ICA took 2.9s.
Using EOG channels: EXG Channel 0, EXG Channel 1
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2500 samples (10.000 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passb

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished


Effective window size : 8.192 (s)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


RuntimeError: No digitization points found.